## Hyperopt

* Objective function to minimize -> `fmin`
* Hyperparameter space -> `hp`
* Search algorithm -> `rand`, `tpe`, `anneal`

In [1]:
# define a model
# define the hyperparameter space
# define the objective function we want to minimize

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, train_test_split

import xgboost as xgb

from hyperopt import hp, rand, fmin, Trials

In [2]:
X, y = load_breast_cancer(return_X_y=True, as_frame=True)
y = y.map({0:1, 1:0})

X.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=3)

X_train.shape, X_test.shape

((398, 30), (171, 30))

In [4]:
# define the hyperparameter space


param_grid = {
    'n_estimators': hp.quniform('n_estimators', 200, 2500, 100),
    'max_depth': hp.uniform('max_depth', 1, 10),
    'learning_rate': hp.uniform('learning_rate', 1e-2, 99e-2),
    'booster': hp.choice('booster', ['gbtree', 'dart']),
    'gamma': hp.quniform('gamma', 0.01, 10, 0.1),
    'subsample': hp.uniform('subsample', 0.5, 0.9),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 0.99),
    'colsample_bylevel': hp.uniform('colsample_bylevel', 0.5, 0.99),
    'colsample_bynode': hp.uniform('colsample_bynode', 0.5, 0.99),
    'reg_lambda': hp.uniform('reg_lambda', 1, 20),
}

In [5]:
# define the objective function


def objective(params):
    params_dict = {
        'n_estimators': int(params['n_estimators']),
        'max_depth': int(params['max_depth']),
        'learning_rate': params['learning_rate'],
        'booster': params['booster'],
        'gamma': params['gamma'],
        'subsample': params['subsample'],
        'colsample_bytree': params['colsample_bytree'],
        'colsample_bylevel': params['colsample_bylevel'],
        'colsample_bynode': params['colsample_bynode'],
    }
    gbm = xgb.XGBClassifier(**params_dict)
    score = cross_val_score(gbm, X_train, y_train, scoring='accuracy', cv=5, n_jobs=-1).mean()
    
    return -score

In [6]:
# randomized search

search = fmin(
    fn=objective,
    space=param_grid,
    max_evals=30,
    algo=rand.suggest,)

100%|██████████| 30/30 [33:56<00:00, 67.87s/trial, best loss: -0.9723417721518987]  


In [7]:
search

{'booster': 1,
 'colsample_bylevel': 0.9226420978115971,
 'colsample_bynode': 0.7937570763390731,
 'colsample_bytree': 0.6625901375082299,
 'gamma': 4.2,
 'learning_rate': 0.3147222215263356,
 'max_depth': 2.2487342046297365,
 'n_estimators': 600.0,
 'reg_lambda': 2.226872713137687,
 'subsample': 0.6992500920524476}

In [11]:
params_dict = {
        'n_estimators': int(search['n_estimators']),
        'max_depth': int(search['max_depth']),
        'learning_rate': search['learning_rate'],
        'booster': int(search['booster']),
        'gamma': search['gamma'],
        'subsample': search['subsample'],
        'colsample_bytree': search['colsample_bytree'],
        'colsample_bylevel': search['colsample_bylevel'],
        'colsample_bynode': search['colsample_bynode'],
    }


In [12]:
model = xgb.XGBClassifier(**params_dict)

model.fit(X_train, y_train)

X_train_preds = model.predict(X_train)
X_test_preds = model.predict(X_test)

print('Train roc_auc: ', accuracy_score(y_train, X_train_preds))

print('Test roc_auc: ', accuracy_score(y_test, X_test_preds))



XGBoostError: [14:04:22] C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0750514818a16474a-1\xgboost\xgboost-ci-windows\src\gbm\gbm.cc:26: Unknown gbm type 1